#### S 0.0 Set parameter and import library

In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
import tensorflow_addons as tfa
import time
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler, MinMaxScaler,Normalizer
from tensorflow.keras.applications.vgg16 import VGG16

In [2]:
import dataset,model_utilis,utils
# from external.yousefi import dataset as external_dataset

In [5]:
# define parameters 
INPUT_SHAPE = (1000,90,1) 
ACTIVATION = None
bn_trainable = True

In [13]:
tf.keras.backend.clear_session()

# 1. Preparation

In [6]:
### function define
lrelu = tf.keras.layers.LeakyReLU
softmax = tf.keras.layers.Softmax
normalization = tfa.layers.InstanceNormalization
identity = tf.keras.layers.Lambda(lambda x:x)

In [7]:
# Gradient reverse layer (Yanin 2015)
@tf.custom_gradient
def grad_reverse(x):
    y = tf.identity(x)
    def custom_grad(dy):
        return -dy
    return y, custom_grad

class GradReverseLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(GradReverseLayer, self).__init__()

    def call(self, x):
        return grad_reverse(x)

In [8]:
def build_model(label_size,normalize=False,seperate=False,latent_shape=None):
    """
    Shallow Convolutional Neural network model, change architecture here
    
    Input
        label_size (tuple): label size
        normalize (bool): add normalization layers 
        seperate (bool): if True return encoder,classifier and discriminator
        latent_shape (None/tuple): return an classifier and discriminator, given seperate == True and latent_shape != None
        
    """
    ### element define
    disc_input  = tf.keras.layers.Input((1000,90,1))
    disc_conv_1 = tf.keras.layers.Conv2D(filters=64, kernel_size=(5,5), strides = (5,5))
    disc_conv_2 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides = (3,3))
    disc_conv_3 = tf.keras.layers.Conv2D(filters=256, kernel_size=(2,2), strides = (2,2))
    disc_flaten = tf.keras.layers.Flatten()
    disc_lab_d1 = tf.keras.layers.Dense(128,activation=lrelu())
    disc_lab_d2 = tf.keras.layers.Dense(label_size)
    disc_dmn_d1 = tf.keras.layers.Dense(128,activation=lrelu())
    disc_dmn_d2 = tf.keras.layers.Dense(1) # change here for categorical crossentropy 
    ### activation
    activation_1 = tf.keras.layers.ReLU()
    activation_2 = tf.keras.layers.ReLU()
    activation_3 = tf.keras.layers.Activation('tanh')
    ### pooling 
    pooling_1 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,1))
    pooling_2 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,1))
    pooling_3 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,1))
    dropout_l = tf.keras.layers.Dropout(0.1)
    dropout_d = tf.keras.layers.Dropout(0.1)
    ### normalization
    disc_norm_c1 = normalization()
    disc_norm_c2 = normalization()
    
    ##### graph define #####
    if normalize == True:
        x = disc_conv_1(disc_input)
        x = disc_norm_c1(x)
        x = activation_1(x)
        x = pooling_1(x)
        x = disc_conv_2(x)
        x = disc_norm_c2(x)
        x = activation_2(x)
        x = pooling_2(x)
        x = disc_conv_3(x)
        x = activation_3(x)
        x = pooling_3(x)
    else:   
        x = disc_conv_1(disc_input)
        #x = disc_norm_c1(x)
        x = activation_1(x)
        x = pooling_1(x)
        x = disc_conv_2(x)
        #x = disc_norm_c2(x)
        x = activation_2(x)
        x = pooling_2(x)
        x = disc_conv_3(x)
        x = activation_3(x)
        x = pooling_3(x)
    p = disc_flaten(x)
    if seperate == True:
        ### encoder
        encoder = tf.keras.models.Model(inputs=disc_input, outputs=p)
        ### shape
        if latent_shape != None:
            lab_input = tf.keras.layers.Input(latent_shape)
            dmn_input = tf.keras.layers.Input(latent_shape)
        else:
            lab_input = tf.keras.layers.Input(p.shape[1:])
            dmn_input = tf.keras.layers.Input(p.shape[1:])
        ### label
        x1 = disc_lab_d1(lab_input)
        x1 = dropout_l(x1)
        o1 = disc_lab_d2(x1)
        lab = tf.keras.models.Model(inputs=lab_input, outputs=o1)
        ### domain 
        x2 = disc_dmn_d1(dmn_input)
        x2 = dropout_d(x2)
        o2 = disc_dmn_d2(x2)
        dmn = tf.keras.models.Model(inputs=dmn_input, outputs=o2)
        return encoder,lab,dmn
    else:
        x1 = disc_lab_d1(p)
        x1 = dropout_l(x1)
        o1 = disc_lab_d2(x1)
        x2 = disc_dmn_d1(p)
        x2 = dropout_d(x2)
        o2 = disc_dmn_d2(x2)
        model = tf.keras.models.Model(inputs=disc_input, outputs=[o1,o2])
        return model

In [9]:
def create_VGG(num=8,top=True):
    base_model = VGG16(include_top=False,input_shape=(1000, 90, 3))
    base_model.trainable = False
    inputs = tf.keras.layers.Input(shape=(1000, 90, 3))
    x = base_model(inputs, training=False)
    p = tf.keras.layers.GlobalAveragePooling2D()(x)
    if top == True:
        p = tf.keras.layers.Dense(128,activation='relu')(p)
        outputs = tf.keras.layers.Dense(num)(p)
        model = tf.keras.Model(inputs, outputs)
    else:
        model = tf.keras.Model(inputs, p)
    return model

In [10]:
### Model 
encoder,classifier,discriminator = build_model(8,normalize=False,seperate=True)
model = model_utilis.define_graph([encoder,classifier],INPUT_SHAPE)

### addition for DANN (gradient reverse)
# grad_reverse_layer = GradReverseLayer()
# dann_domain_discrimination_graph = model_utilis.define_graph([encoder,grad_reverse_layer,discriminator],INPUT_SHAPE)

### VGG 
# vgg_top = create_VGG(8,False)
# _,vgg_clf,vgg_discriminator = build_model(8,normalize=False,seperate=True,latent_shape=vgg_top.output_shape[1:])

# vgg_model = model_utilis.define_graph([vgg_top,vgg_clf],(1000,90,3))
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1000, 90, 1)]     0         
_________________________________________________________________
model (Model)                (None, 1024)              206848    
_________________________________________________________________
model_1 (Model)              (None, 8)                 132232    
Total params: 339,080
Trainable params: 339,080
Non-trainable params: 0
_________________________________________________________________


#### S1.3 Data

In [ ]:
# External dataset (Yousefi,2017)

folderpath = "./external/yousefi/Dataset/Data"
df = external_dataset.import_dataframe(folderpath)

In [ ]:
# DatasetObject EXT

dataset_exp1 = external_dataset.DatasetObject(df,cnn=True,sep=None)

In [ ]:
del df

In [10]:
# Self dataset EXP1 

folderpath1 = "./data/exp_1"  # CHANGE THIS IF THE PATH CHANGED
df_exp1 = dataset.import_dataframe(folderpath1) # CHANGE THIS IF THE PATH CHANGED
# processing step, required
df_exp1 = df_exp1.drop([f"theta_{i}" for i in range(1,91)], axis=1) 
print(df_exp1.shape)

dataset_exp1 = dataset.DatasetObject(df_exp1,cnn=True,stacking=False)
ohe_y = dataset_exp1.one_hot(1)
ohe_z = dataset_exp1.one_hot(2)

Found 80 files.
input_user10_bendfwd.csv annotation_user10_bendfwd.csv user10
input_user10_kneel.csv annotation_user10_kneel.csv user10
input_user10_lie.csv annotation_user10_lie.csv user10
input_user10_sit.csv annotation_user10_sit.csv user10
input_user10_sitrotate.csv annotation_user10_sitrotate.csv user10
input_user10_stand.csv annotation_user10_stand.csv user10
input_user10_standrotate.csv annotation_user10_standrotate.csv user10
input_user10_walking.csv annotation_user10_walking.csv user10
input_user1_bendfwd.csv annotation_user1_bendfwd.csv user1
input_user1_kneel.csv annotation_user1_kneel.csv user1
input_user1_lie.csv annotation_user1_lie.csv user1
input_user1_sit.csv annotation_user1_sit.csv user1
input_user1_sitrotate.csv annotation_user1_sitrotate.csv user1
input_user1_stand.csv annotation_user1_stand.csv user1
input_user1_standrotate.csv annotation_user1_standrotate.csv user1
input_user1_walking.csv annotation_user1_walking.csv user1
input_user2_bendfwd.csv annotation_user2

In [ ]:
# Self dataset EXP2 


folderpath2 = "./data/exp_2" # CHANGE THIS IF THE PATH CHANGED
df_exp2 = dataset.import_dataframe(folderpath2)
# Preprocessing 
df_exp2['user'] = df_exp2['user'].map(lambda x: x.split('.')[0]) # processing step, required
df_exp2 = df_exp2.iloc[:,2:] # processing step, required
print(df_exp2.shape)

dataset_exp2 = dataset.DatasetObject(df_exp2,cnn=True,stacking=False)
ohe_y = dataset_exp2.one_hot(1,ohe_y)
ohe_z = dataset_exp2.one_hot(2,ohe_z)

In [ ]:
df_ = pd.concat([df_exp1,df_exp2],axis=0)
dataset_exp4 = dataset.DatasetObject(df_,cnn=True,stacking=False)
ohe_y = dataset_exp4.one_hot(1)
ohe_z = dataset_exp4.one_hot(2)

In [ ]:
del df_exp1, df_exp2

# 2. Experiment

In [17]:
tf.random.set_seed(1234)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=200)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [20]:
train[0].shape

(840, 1000, 90, 1)

In [18]:
### single
train,test = dataset_exp1.query([6])
history = model.fit(x=train[0],y=train[1],batch_size=64,epochs=100,verbose=1,validation_data=(test[0],test[1]),callbacks=[callback])

### cross validation
# model,histories,scores = model_utilis.cross_validation(model,dataset_exp1,batch_size=64,epochs=10)

[0, 1, 2, 3, 4, 5, 7, 8, 9]
Epoch 1/100
14/14 [==============================] - 1s 61ms/step - loss: 3.8036 - accuracy: 0.1298 - val_loss: 9.0686 - val_accuracy: 0.1250
Epoch 2/100
14/14 [==============================] - 1s 56ms/step - loss: 3.6468 - accuracy: 0.1119 - val_loss: 2.1017 - val_accuracy: 0.1154
Epoch 3/100
14/14 [==============================] - 1s 57ms/step - loss: 2.1038 - accuracy: 0.1369 - val_loss: 2.1097 - val_accuracy: 0.0962
Epoch 4/100
14/14 [==============================] - 1s 57ms/step - loss: 2.0915 - accuracy: 0.1357 - val_loss: 2.0916 - val_accuracy: 0.1058
Epoch 5/100
14/14 [==============================] - 1s 57ms/step - loss: 2.0795 - accuracy: 0.1571 - val_loss: 2.0878 - val_accuracy: 0.1058
Epoch 6/100
14/14 [==============================] - 1s 57ms/step - loss: 2.0772 - accuracy: 0.1440 - val_loss: 2.0887 - val_accuracy: 0.1923
Epoch 7/100
14/14 [==============================] - 1s 57ms/step - loss: 2.0775 - accuracy: 0.1381 - val_loss: 2.0837 -

In [ ]:
# evaluation
train,test = dataset_exp1.query([6])
cmtx = utils.evaluation(model,test[0],test[1],ohe=ohe_y)

# 3. Save

In [ ]:
current_time = datetime.now().strftime("%Y%m%d%H%M%S")
model_name = f"model_{current_time}"
savepath = f'./saved_model/{model_name}.h5'
model.save(savepath)
print("model saved: ",savepath)

In [ ]:
cmtx.to_csv(f"./record/cmtx_{model_name}.csv")